# Problemas de alocação

Os problemas de alocação modelam uma relação entre dois tipos de entidades: *compromissos* e *recursos*. Adicionalmente, as restrições sobre a alocação de recursos a compromissos classificam-se em dois tipos: *obrigações* e *limitações*.

Nesta aula vamos considerar um problema de alocação. Pretende-se que faça a modelação do problema em Programação Inteira e que use o SCIP para os resolver.

Usaremos a biblioteca [OR-Tools](https://developers.google.com/optimization) para fazer a interface para o SCIP. Esta biblioteca pode ser instalada com o commando `pip install ortools`.


In [ ]:
!pip install ortools

     |████████████████████████████████| 14.5 MB 92 kB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
     |████████████████████████████████| 132 kB 65.6 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.2.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.


## Horário de um centro de estudos

Um centro de estudos possui $S$ salas de aula, $P$ professores, e está aberto $H$ horas por dia, durante $D$ dias na semana.

O centro de estudos funciona com as seguintes regras:
- Cada professor não pode dar mais do que $C$ horas por dia.
- Todos os professores do centro devem ter uma carga horária semanal parecida (diferença máxima de 2 horas).
- Não é permitido mais do que um professor por sala.
- Alocações contíguas do mesmo professor têm de ser na mesma sala.

Pretende-se estabelecer um horário para o centro de estudos que permita ter um número mínimo de aulas $N$ por semana.

### Análise do problema

Este é um problema de alocação. Pretende-se alocar professores a salas de aula, ao longo da semana, sendo o tempo de ocupação das salas de uma hora. 

Existem $P$ professores, que podemos identificar por um índice $p \in [0..P\!-\!1]$, e podemos identificar cada sala disponível num dado dia, a uma dada hora, por um triplo $(s,d,h) \in [0..S\!-\!1]\times[0..D\!-\!1]\times[0..H\!-\!1]$.

Vamos usar uma família $x_{p,s,d,h}$ de variáveis binárias (i.e., que assumem valores inteiros $\{0,1\}$), com a seguinte semântica

$$x_{p,s,d,h} == 1  \quad \mbox{se e só se} \quad \mbox{o professor $p$ for alocado à sala $s$, no dia $d$, à hora $h$.}$$

Estas $P\times S\times D\times H$ variáveis são convenientemente representadas numa matriz $X$ instanciável com valores $\{0,1\}^{P\times S\times D\times H}$, a que se costuma chamar *matriz de alocação*.

Destaca-se ainda o seguinte:

**Limitações** (que impõem limites máximos à alocação)

1. A carga horária diária máxima de cada professor é $C$.
2. Cada sala tem alocado, no máximo, um professor.
3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.
4. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.


**Obrigações** (que impõem limites mínimos à alocação)

5. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).
6. O número mínimo de aulas por semana é $N$.

### Implementação

Começamos por importar a biblioteca de programação linear do OR-Tools e criar uma instância do *solver*.


Depois inicializamos o *solver* `horario` e definir os valores para as constantes $S$, $P$, $H$, $D$, $C$ e $N$.

In [ ]:
from ortools.linear_solver import pywraplp

def horario(S,P,H,D,C,N):
  solver = pywraplp.Solver.CreateSolver('SCIP')

  #Exercicio 1
  x = {}

  for p in range(P): #Para cada professor
    for s in range(S):  #Para cada sala
      for h in range(H):  #Para cada hora
        for d in range(D):  #Para cada dia
          x[p,s,d,h] = solver.BoolVar("x[%d,%d,%d,%d]" % (p,s,d,h))

  T = solver.IntVar(0.0,H*D,"T")
  for d in range(D):
    for p in range(P):
       solver.Add(sum([x[p,s,d,h] for h in range(H) for s in range(S)]) <= C)


  #Exercicio 2
  
  #Restriçao 2.
  for s in range(S):
    for d in range(D):
      for h in range(H):
        solver.Add(sum([x[p,s,d,h] for p in range(P)]) <= 1)
  
  #Restrição 3.
  for d in range(D):
    for h in range(H):
      for p in range(P):
        solver.Add(sum([x[p,s,d,h] for s in range(S)]) <= 1)
  
  #Restrição 4.
  for p in range(P):
    solver.Add(sum(x[p,s,d,h] for s in range(S) for d in range(D) for h in range(H)) <= T)
    solver.Add(sum(x[p,s,d,h] for s in range(S) for d in range(D) for h in range(H)) <= T)

  #Restrição 6.
  solver.Add(sum(x.values()) >= N)

  #Restrição 5.
  for o in range(P):
    for d in range(D):
      for h in range(H-1):
        for s in range(s):
          for s_ in range(s+1,S):
            solver.Add(x[p,s,d,h] + x[p,s_,d,h+1] <= 1)
            solver.Add(x[p,s_,d,h] + x[p,s,d,h+1] <= 1) 
            


  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
    for p in range(P):
      print("Prof",p)
      for h in range(H):
        print("Hora",h,end='')
        for d in range(D):
          print(",",end='')
          for s in range(S):
            if round(x[p,s,d,h].solution_value()) == 1:
              print(s,end='')
        print()


#S, P, H, D, C, N = 1, 2, 6, 5, 4, 
x = horario(1,2,6,5,4,21)



Prof 0
Hora 0,0,0,0,0,0
Hora 1,0,0,0,0,0
Hora 2,0,0,0,0,0
Hora 3,0,0,0,0,0
Hora 4,,,,,
Hora 5,,,,,
Prof 1
Hora 0,,,,,
Hora 1,,,,,
Hora 2,,,,,
Hora 3,,,,,
Hora 4,0,0,0,0,0
Hora 5,0,0,0,0,0


Em seguida, declaramos a matriz de alocação, $X$, e uma variável $T$ que irá representar a carga horária semanal mínima de cada professor (com as restrições adequadas).

### Exercício 1

Complete a declaração da matriz de alocação $X$ como um dicionário.

In [ ]:
x = {}
# completar

                
def X(p,s,d,h):              # abreviatura
    return x[p][s][d][h]           

T = horario.IntVar(0.0,H*D,"T")

Passamos agora à modelação das restrições e à sua introdução no *solver*.

A restrição

1. A carga horária diaria máxima de cada professor é $C$

pode expressar-se da seguinte forma:

$$\forall_{d< D} \cdot \forall_{p< P} \cdot \quad \sum_{h< H,\,s< S} x_{p,s,d,h} \leq C$$

In [ ]:
for d in range(D):
    for p in range(P):
       solver.Add(sum([X(p,s,d,h) for h in range(H) for s in range(S)]) <= C)

### Exercício 2

Apresente as fórmulas que modelam as restantes restrições e acrescente-as ao problema `horario`.

2. Cada sala tem alocada, no máximo, um professor.
$$\forall_{d} \forall_{h} \forall_{s} \cdot \sum_{p} x{p,s,d,h} \leq 1 $$


In [ ]:
# completar

3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.

$$
\forall_{d} \forall_{h} \forall_{p} \cdot \sum_{s} x{p,s,d,h} \leq 1
$$

In [ ]:
# completar

4. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).

$$
\forall_{s} \forall_{h} \cdot \sum_{p} x{p,s,h} \leq 
$$

In [ ]:
# completar

5. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.

$$
\forall_{p} \forall_{d} \forall_{} \cdot \sum_{p} x{p,s,h} \leq 
$$

In [ ]:
# completar

6. O número mínimo de aulas por semana é $N$.

$$
...
$$

In [ ]:
# completar

### Exercício 3

Finalize a resolução do problema procurando uma solução, e imprimindo o número de aulas por semana, caso exista.

In [ ]:
# completar

### Exercício 4

Defina funções para construir os horarios de professores e salas individuais, e para apresentar de forma legível esses horários.

In [ ]:
# completar

### Exercício 5

Explore o comportamento do modelo pela variação dos parâmetros $S$, $P$, $H$, $D$, $C$ e $N$.

### Exercício 6

Queremos agora acrescentar a seguinte regra no funcionamento do centro de estudos:

> Cada professor tem de ter um dia da semana em que não dá aulas.

Esta *obrigação* poderia ser expressa matematicamente, de forma direta, por
$$
\forall_{p<P}.\exists_{d<D}. \quad \sum_{s<S,h<H} x_{p,s,d,h} = 0
$$
ou, em alternativa, pela seguinte expressão
$$
\forall_{p<P}. \quad \bigvee_{d<D} \big(\sum_{s<S,h<H} x_{p,s,d,h} = 0\big) 
$$

Contudo a disjunção não tem uma representação direta nos solvers para programação inteira. Para a implementar podemos acrescentar uma família de variáveis binárias $y_{p,d}$ que indicam se o professor $p$ dá aulas no dia $d$, com a seguinte restrição que limita o número máximo de dias em que o professor dá aulas.

$$\forall_{p<P} \cdot \quad \sum_{d<D} y_{p,d} \leq D-1 $$

O valor de  $y_{p,d}$ deve também de alguma forma limitar superiormente as aulas que o professor $p$ dá no dia $d$.
Apresente uma fórmula que modele esta nova limitação.

$$...$$



Acrescente agora estas fórmulas ao problema `horario`, e encontre nova solução.

In [ ]:
# completar